In [1]:
# read data from data set
from utils.data_utils import jigsaw_toxix_ds_get_df
import numpy as np
from config import *
df = jigsaw_toxix_ds_get_df()
comments = df["comment_text"].tolist()
comments = [x.replace('\n', ' ') for x in comments]
dump_text = '\n'.join(comments)
total_classes = 6
class_matrix = df[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]].astype('int')
label_matrix = class_matrix.values



In [2]:
# load word2vec model
import os 
from config import model_folder
from model_factory.embeddings.w2v import w2v_load_from_keyedvectors, build_embedding_layer
w2v_name = 'google_keyed_vector_format.bin'
model_path = os.path.join(model_folder, w2v_name)
    
w2v_model = w2v_load_from_keyedvectors(model_path)
vocab = w2v_model.vocab

word_2_idx = dict(zip(vocab.keys(), range(len(vocab))))
max_sent_length = 80
trainable = False
embedding_layer = build_embedding_layer(w2v_model, word_2_idx, len(vocab), max_sent_length, trainable)
print(embedding_layer)

Using TensorFlow backend.


In [6]:
# tokenization
import nltk
from keras.preprocessing.sequence import pad_sequences



def tokenizer(text, word2idx, max_len=80, total=5000):
    text = text[:total]
    for sentence_idx in range(len(text)):
        sentence = text[sentence_idx]
        text[sentence_idx] = nltk.word_tokenize(sentence)
 
    def _sent_to_idx(s, w2i):
        for word_idx in range(len(s)):
            word = s[word_idx]
            idx = w2i.get(word, w2i['null'])
            s[word_idx] = idx
        return s
        
    for sentence_idx in range(len(text)):
        sentence = text[sentence_idx]
        sequence = _sent_to_idx(sentence, word2idx)
        text[sentence_idx] = sequence
    
    text = pad_sequences(text, maxlen=max_len, value=word2idx['null'])   
    return text

num_samples = 5000
tokenized_sequence = tokenizer(comments, word_2_idx, max_sent_length, total=num_samples)
label_matrix = label_matrix[:num_samples]

In [9]:
# build model
from keras import models, layers, objectives, metrics
h_dim = 300
m = models.Sequential()
m.add(embedding_layer)
m.add(layers.LSTM(
    units=h_dim, 
    return_sequences=False,
))
m.add(layers.Dense(total_classes, activation='sigmoid'))
m.compile(optimizer='adam', loss=objectives.binary_crossentropy, metrics=[metrics.top_k_categorical_accuracy])


In [10]:
# join embeddings with data set dataframe
m.fit(tokenized_sequence, label_matrix)


Epoch 1/1


  32/5000 [..............................] - ETA: 1:49 - loss: 0.7036 - top_k_categorical_accuracy: 0.9688

  64/5000 [..............................] - ETA: 1:01 - loss: 0.6756 - top_k_categorical_accuracy: 0.9844

  96/5000 [..............................] - ETA: 44s - loss: 0.6479 - top_k_categorical_accuracy: 0.9896 

 128/5000 [..............................] - ETA: 36s - loss: 0.6088 - top_k_categorical_accuracy: 0.9922

...

4896/5000 [============================>.] - ETA: 0s - loss: 0.1652 - top_k_categorical_accuracy: 0.9996

4928/5000 [============================>.] - ETA: 0s - loss: 0.1655 - top_k_categorical_accuracy: 0.9996

4960/5000 [============================>.] - ETA: 0s - loss: 0.1653 - top_k_categorical_accuracy: 0.9996

4992/5000 [============================>.] - ETA: 0s - loss: 0.1651 - top_k_categorical_accuracy: 0.9996

5000/5000 [==============================] - 15s 3ms/step - loss: 0.1654 - top_k_categorical_accuracy: 0.9996


In [ ]:
# extract embedding column and label columns

In [ ]:
# convert to X, Y

In [ ]:
# fit model and test